# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)
---
## 

## CRUD pour verifier les donneés

In [145]:
# import
import datetime
import pandas as pd
from sqlalchemy import and_, or_

from sqlalchemy import  create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func

from components.models import Client, DonnePersonnel, Commande, Produit, Genre, Promotion, Age, Region, Platform, Publisher, Year

In [79]:
#db_reader.py

engine = create_engine("sqlite:///BD_Ventes_de_jeux_video.bd") 

# session
Session = sessionmaker(bind=engine)
session = Session()

## C - CREATE
- create_client(age_id, region_id)
- create_donne_personnel(login, mot_de_passe_hash)
- create_commande(client_id, produit_id, nb_produit)
- create_promotion(produit_id:int, promotion_percent:int, region_id_promo:int)



In [80]:
# Function pour creater un Client, Commande
def create_client(age_id:int, region_id:int):
    """Create and persist a new Client in the database.

    Args:
        age_id (int): ID of the age category for the client.
        region_id (int): ID of the region where the client belongs.

    Returns:
        Client: The newly created Client object.

    Raises:
        Exception: If the session commit fails, the transaction is rolled back and the exception is re-raised.
    """
    try:
        client = Client(
            age_id=age_id,
            region_id=region_id
        )
        session.add(client)
        session.commit()
        return client
    except Exception as e:
        session.rollback()
        raise e
    
def create_donne_personnel(login, mot_de_passe_hash):
    """Create and persist a new DonnePersonnel (personal data) record for a client.

    Args:
        login (str): Login username for the personal data.
        mot_de_passe_hash (str): Hashed password.

    Returns:
        DonnePersonnel: The newly created DonnePersonnel object.

    Raises:
        Exception: If the session commit fails, the transaction is rolled back and the exception is re-raised.
    """
    try:
        donne = DonnePersonnel(
            login=login,
            mot_de_passe_hash=mot_de_passe_hash,
            date_suppression=None,
            anonymise=False
        )
        session.add(donne)
        session.commit()
        return donne
    except Exception as e:
        session.rollback()
        raise e

def create_commande(client_id, produit_id, nb_produit):
    """Create and persist a new order (Commande) in the database, optionally applying a promotion.

    The function looks for a promotion associated with the given product — 
    if found, the order will reference the promotion; otherwise, no promotion is applied.

    Args:
        client_id (int): ID of the client placing the order.
        produit_id (int): ID of the product being ordered.
        nb_produit (int): Number of units ordered.

    Returns:
        Commande: The newly created Commande object.

    Raises:
        Exception: If the session commit fails. The session will be rolled back and the exception re-raised.
    """
    try:
        promo = session.query(Promotion).filter(Promotion.produit_id=={produit_id}).first()
        promo_id = promo.promotion_id if promo else 0
        session.add(Commande(
            client_id = client_id,
            produit_id = produit_id,
            nb_produit = nb_produit,
            promotion_id = promo_id
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    
def create_promotion(produit_id:int, promotion_percent:int, region_id_promo:int):
    """Create a promotion for a given product and link it to a region.

    Args:
        produit_id (int): ID of the product.
        promotion_percent (int): Discount percent.
        region_id_promo (int): ID of the region for this promotion.

    Raises:
        Exception: If commit fails, the session is rolled back and the exception re-raised.
    """
    try:
        region = session.query(Region).filter(Region.region_id==region_id_promo).first()

        obj = Promotion(
            produit_id = produit_id,
            promotion_percent = promotion_percent
        )
        obj.regions.append(region)
        session.add(obj)
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

# R - READ
- read_table(table_class, limit=None, filter_exp=None) -- jeneral pour verifier
- read_promo(limit=None, filter_exp=None)
- read_produit(limit=None, filter_exp=None)
- read_command(limit=None, filter_exp=None)
- read_client(limit=None, filter_exp=None)

In [165]:
# Function pour faire un roquet et pour retourner DataFrame
def read_table(table_class, limit=None, filter_exp=None):
    """
    table_class: class SQLAlchemy (par exemple.: Client, Commande)
    filter: opt
    limit: opt

    """
    query = session.query(table_class)

    
    if table_class is Client:
        query.update(
        {Client.date_derniere_utilisation: func.now()},
        synchronize_session=False
        )
        session.commit()

    if filter_exp is not None:
        query = query.filter(filter_exp)
    
    if limit is not None:
        query = query.limit(limit)
    
    df = pd.read_sql(query.statement, session.bind)
    return df

def read_promo(limit=None, filter_exp=None):
    query = (session.query(
        Promotion,
        Produit.name
        )).join(Produit, Produit.produit_id == Promotion.produit_id)
    
    if filter_exp is not None:
        query = query.filter(filter_exp)
    if limit is not None:
        query = query.limit(limit)

    df = pd.read_sql(query.statement, session.bind)
    
    list_reg = ""
    for promo, prod_name in query:
        for reg in promo.regions:
            list_reg += f"{promo.promotion_percent}% apply to {prod_name}: {reg.region_nom}\n"
 
    
    return df, list_reg

def read_produit(limit=None, filter_exp=None):
    query = (
        session.query(
            Produit.produit_id, Produit.prix, Produit.name,
            Year.year_nom.label("year_nom"),
            Platform.platform_nom.label("platform_nom"),
            Genre.genre_nom.label("genre_nom"),
            Publisher.publisher_nom.label("publisher_nom")
        )
        .join(Year, Year.year_cod == Produit.year_n)
        .join(Platform, Platform.platform_cod == Produit.platform_cod)
        .join(Genre, Genre.genre_cod == Produit.genre_cod)
        .join(Publisher, Publisher.publisher_cod == Produit.publisher_cod)
    )
    
    if filter_exp is not None:
        query = query.filter(filter_exp)
    if limit is not None:
        query = query.limit(limit)
    
    df = pd.read_sql(query.statement, session.bind)

    return df


def read_command(limit=None, filter_exp=None):
    query = (session.query(
        Commande.commande_id,
        Commande.nb_produit,
        Promotion.promotion_percent,
        Produit.name.label("produit_nom"),
        )
    .join(Produit, Produit.produit_id == Commande.produit_id)
    .outerjoin(Promotion, Promotion.promotion_id == Commande.promotion_id)
    )
    
    if filter_exp is not None:
        query = query.filter(filter_exp)
    if limit is not None:
        query = query.limit(limit)

    df = pd.read_sql(query.statement, session.bind)

    return df

def read_client(limit=None, filter_exp=None):
    """
    Récupère les informations des clients avec les données associées et le nombre de commandes.

    Args:
        limit (int, optional): Nombre maximum de clients à récupérer. Par défaut, None = tous.
        filter_exp (Expression, optional): Expression SQLAlchemy pour filtrer les clients.
                                           Exemple : Client.region_id == 0

    Behavior:
        - Joint les tables Age et Region pour récupérer les informations associées.
        - Calcule le nombre de commandes par client.
        - Applique un filtre et une limite si fournis.
        - Retourne le résultat sous forme de DataFrame pandas.

    Returns:
        pandas.DataFrame: Contient les colonnes suivantes :
            - client_id
            - age_plage
            - region_nom
            - Nb_commande
    """
    query = ((session.query(
        Client.client_id,
        Age.age_plage,
        Region.region_nom,
        func.count(Commande.commande_id).label("Nb_commande")
        )).join(Commande, Commande.client_id == Client.client_id).join(Age, Age.age_id == Client.age_id).join(Region, Region.region_id == Client.region_id)).group_by(Client.client_id)
    
    if filter_exp is not None:
        query = query.filter(filter_exp)
    
    if limit is not None:
        query = query.limit(limit)  

    session.commit()
    
    df = pd.read_sql(query.statement, session.bind)
    return df

def update_client_date(client_id):
    obj = session.get(Client, client_id)
    if obj is None:
        return None
    setattr(obj, "date_derniere_utilisation", datetime.datetime.now())
    session.commit()


In [164]:
session.query(Promotion).filter(Promotion.promotion_percent == 50).all()

In [163]:
read_command(limit=5, filter_exp=Promotion.promotion_percent == 50)

,commande_id,nb_produit,promotion_percent,produit_nom


In [113]:
df_promo = read_promo(limit=3)
print(df_promo[0])
print(df_promo[1])


   promotion_id  promotion_percent  produit_id  \
0             1                 70        5511   
1             2                 50        6512   
2             3                 50        6564   

                                      name  
0        Hatsune Miku: Project Diva Extend  
1  Konjiki no Gashbell!! Makai no Bookmark  
2               Oddworld: Stranger's Wrath  
70% apply to Hatsune Miku: Project Diva Extend: NA
50% apply to Konjiki no Gashbell!! Makai no Bookmark: Other
50% apply to Oddworld: Stranger's Wrath: JP



In [116]:
read_produit( filter_exp=(Publisher.publisher_nom == "Nintendo"), limit=5)

,produit_id,prix,name,year_nom,platform_nom,genre_nom,publisher_nom
0,16,111,Kinect Adventures!,2007.0,DS,Puzzle,Nintendo
1,44,103,Halo 3,1999.0,DS,Misc,Nintendo
2,63,42,Halo: Reach,2007.0,DS,Misc,Nintendo
3,66,128,Halo 4,2015.0,DS,Misc,Nintendo
4,73,114,Minecraft,2010.0,DS,Puzzle,Nintendo


In [157]:
read_client(filter_exp=Region.region_nom == 'NA')

,client_id,age_plage,region_nom,Nb_commande
0,1,7 - 14 ans,NA,11
1,9,0 - 6 ans,NA,16
2,11,0 - 6 ans,NA,15
3,12,7 - 14 ans,NA,19
4,17,0 - 6 ans,NA,11
...,...,...,...,...
92,327,7 - 14 ans,NA,18
93,328,7 - 14 ans,NA,13
94,331,0 - 6 ans,NA,19
95,343,0 - 6 ans,NA,14


# U - UPDATE
- update_table

In [ ]:
def update_table(table_nom, data_id, **kwargs):
    """
    Met à jour les colonnes spécifiées d'un enregistrement dans une table SQLAlchemy.

    Args:
        table_nom (DeclarativeMeta): La classe SQLAlchemy représentant la table.
        data_id (int): L'identifiant de l'enregistrement à mettre à jour.
        **kwargs: Paires clé-valeur représentant les colonnes à modifier et leurs nouvelles valeurs.
                  Exemple : age_id=1, region_id=0

    Behavior spécifique:
        - Si la table est `Client`, la colonne `date_derniere_utilisation` sera mise à jour avec l'heure actuelle.

    Returns:
        None si l'objet avec `data_id` n'existe pas. Sinon, commit les changements dans la base de données.

    Exemple:
        update_table(Client, 51, age_id=1, region_id=0)
    """
    obj = session.get(table_nom, data_id)
    if not obj:
        return None
    
    for field, value in kwargs.items():
        setattr(obj, field, value)
    

    if table_nom is Client:
        obj.date_derniere_utilisation = func.now()
        
    session.commit()
    print(f"L’enregistrement dans {table_nom.__tablename__} a été renouvelé.")



### Vérifier le bon fonctionnement

In [141]:
read_client(filter_exp=(Client.client_id == 51))

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,51,1,2,2025-12-01 13:31:37,2025-12-02 14:00:04


In [143]:
data = {
    'age_id': 1,
    'region_id': 0
}
update_table(Client, 51, **data)

L’enregistrement dans clients a été renouvelé.


In [144]:
read_client(filter_exp=(Client.client_id == 51))

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,51,1,0,2025-12-01 13:31:37,2025-12-02 14:03:33


# D - DELETE
- delete_object()
- delete_filtre()

In [ ]:
def delete_objet(table_nom, data_id):
    """
    Supprime un enregistrement spécifique d'une table SQLAlchemy.

    Args:
        table_nom (DeclarativeMeta): La classe SQLAlchemy représentant la table.
        data_id (int): L'identifiant de l'enregistrement à supprimer.

    Behavior:
        - Cherche l'objet dans la base via `session.get`.
        - Si l'objet existe, le supprime et commit la transaction.
        - Si l'objet n'existe pas, aucune suppression n'est effectuée.
        - Capture les exceptions et affiche un message d'erreur.

    Returns:
        None

    Exemple:
        delete_objet(Client, 51)
    """
    try:
        obj = session.get(table_nom, data_id)
        if obj is not None:
            session.delete(obj)
        session.commit()
    except Exception as e:
        print(e)

def delete_filtre(table_nom, filter_exp):
    """
    Supprime tous les enregistrements d'une table SQLAlchemy correspondant à un filtre donné.

    Args:
        table_nom (DeclarativeMeta): La classe SQLAlchemy représentant la table.
        filter_exp: Expression de filtre SQLAlchemy pour sélectionner les enregistrements à supprimer.
                    Exemple : Client.age_id == 1

    Behavior:
        - Crée une requête avec `session.query(table_nom).filter(filter_exp)`.
        - Supprime tous les enregistrements filtrés via `.delete()`.
        - Capture les exceptions et affiche un message d'erreur.

    Returns:
        None

    Exemple:
        delete_filtre(Client, Client.age_id == 1)
    """
    try:
        query = session.query(table_nom).filter(filter_exp)
        query.delete(synchronize_session=False)
        session.commit()

    except Exception as e:
        print(e)

# CLOSE ALL

In [ ]:
# session.rollback()

In [77]:
session.close()
engine.dispose()